In [ ]:
#Étape 1 : Chargement et nettoyage des données

In [8]:
import pandas as pd
import numpy as np

train_path = "C:/Users/fbe/Desktop/Bloc 4/application_train.csv"
test_path = "C:/Users/fbe/Desktop/Bloc 4/application_test.csv"

# --- Chargement ---
dt1 = pd.read_csv(train_path, sep=";", low_memory=False)
test = pd.read_csv(test_path, sep=";", low_memory=False)
print(" Chargement terminé :", dt1.shape, test.shape)

# Nettoyage valeur aberrante
dt1["DAYS_EMPLOYED"] = dt1["DAYS_EMPLOYED"].replace(365243, np.nan)

 Chargement terminé : (307511, 122) (48744, 121)


In [9]:
print(dt1.columns.tolist())


['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELE

In [10]:
# RGPD - Minimisation des données
# Seules les variables nécessaires à l’évaluation du risque de crédit sont conservées.
# Aucune donnée personnelle directement identifiable (nom, adresse, téléphone, email, etc.) n’est utilisée.

In [11]:
cols_utiles = [
    'SK_ID_CURR',
    'CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE',
    'OCCUPATION_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
    'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
    'CNT_CHILDREN', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'TARGET'
]


data = dt1[cols_utiles].copy()
print(" Variables retenues :", len(data.columns))

 Variables retenues : 20


In [ ]:
# Garder uniquement les colonnes nécessaires 
data = dt1[ [c for c in cols_utiles if c in dt1.columns] ].copy()
print("Variables retenues :", data.shape[1], "colonnes ->", data.columns.tolist())

Variables retenues : 20 colonnes -> ['SK_ID_CURR', 'CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'CNT_CHILDREN', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'TARGET']


In [ ]:
# ============================
# - Identifier colonnes numériques entièrement négatives
# ============================
col_neg = [
    col for col in data.select_dtypes(include=np.number).columns
    if (data[col].dropna() <= 0).all()
]

if col_neg:
    data_abs = data[col_neg].copy().abs()   # appliquer la valeur absolue
    data = data.drop(columns=col_neg)
    data = pd.concat([data_abs, data], axis=1)
    print(f" Colonnes corrigées (abs) : {col_neg}")
    print(data_abs.info())
else:
    print(" Aucune colonne entièrement négative détectée.")

# ============================

🔧 Colonnes corrigées (abs) : ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   DAYS_BIRTH       307511 non-null  int64  
 1   DAYS_EMPLOYED    252137 non-null  float64
 2   DAYS_ID_PUBLISH  307511 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 7.0 MB
None


In [ ]:
# ============================
#  Définition des colonnes numériques et catégorielles
# ============================

cat_cols = [
    'CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE',
    'OCCUPATION_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'
]

num_cols = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
    'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
    'CNT_CHILDREN', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'
]

# ============================
#  Nettoyage complet des nombres 
# ============================

for c in num_cols:
    if c in data.columns:
        # Remplacer virgule par point
        data[c] = data[c].astype(str).str.replace(",", ".", regex=False)
        # Supprimer tout sauf chiffres, point, signe moins et exposants
        data[c] = data[c].str.replace(r"[^0-9\.\-eE]", "", regex=True)
        # Convertir en float (NaN si conversion impossible)
        data[c] = pd.to_numeric(data[c], errors='coerce')

# ============================
#  Vérification rapide des valeurs encore non convertibles
# ============================

for c in num_cols:
    non_convertibles = data[c][data[c].isna()]
    if len(non_convertibles) > 0:
        print(f" Colonne {c} a {len(non_convertibles)} valeurs non convertibles")

# ============================
#  - Gestion des NaN
# ============================

data[cat_cols] = data[cat_cols].fillna("Unknown")
data[num_cols] = data[num_cols].fillna(data[num_cols].mean())

print(" NaN remplis : catégorielles -> 'Unknown', numériques -> moyennes")

⚠️ Colonne AMT_ANNUITY a 12 valeurs non convertibles
⚠️ Colonne AMT_GOODS_PRICE a 278 valeurs non convertibles
⚠️ Colonne DAYS_EMPLOYED a 55374 valeurs non convertibles
⚠️ Colonne EXT_SOURCE_1 a 173378 valeurs non convertibles
⚠️ Colonne EXT_SOURCE_2 a 660 valeurs non convertibles
⚠️ Colonne EXT_SOURCE_3 a 60965 valeurs non convertibles
✅ NaN remplis : catégorielles -> 'Unknown', numériques -> moyennes


In [ ]:
import scipy.sparse
from sklearn.preprocessing import OneHotEncoder

# ============================
#  - Encodage One-Hot
# ============================

encoder = OneHotEncoder(
    drop='first',
    sparse_output=True,
    handle_unknown='ignore'
)

X_cat = encoder.fit_transform(data[cat_cols])

# ============================
# - Préparation des colonnes numériques
# ============================


for c in num_cols:
    data[c] = pd.to_numeric(data[c], errors='coerce')

# Remplir les NaN avec la moyenne
X_num = data[num_cols].fillna(data[num_cols].mean())
X_num_sparse = scipy.sparse.csr_matrix(X_num.to_numpy())

# ============================
#  - Fusion finale des données
# ============================

X_final = scipy.sparse.hstack([X_num_sparse, X_cat])
y = data['TARGET']

print(" Encodage terminé.")
print(" Dimensions finales :", X_final.shape)

✅ Encodage terminé.
🧩 Dimensions finales : (307511, 43)


In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, stratify=y, random_state=42
)
print(" Split effectué :", X_train.shape, X_test.shape)

✅ Split effectué : (246008, 43) (61503, 43)


In [ ]:
from imblearn.over_sampling import SMOTE

# Création du suréchantillonneur
smote = SMOTE(random_state=42)

# Application uniquement sur les données d’entraînement
X_res, y_res = smote.fit_resample(X_train, y_train)

print("Avant SMOTE :", y_train.value_counts().to_dict())
print("Après SMOTE  :", y_res.value_counts().to_dict())
print(" Jeu d'entraînement équilibré :", X_res.shape)

Avant SMOTE : {0: 226148, 1: 19860}
Après SMOTE  : {0: 226148, 1: 226148}
✅ Jeu d'entraînement équilibré : (452296, 43)


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Entraînement avec les données équilibrées
xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    tree_method='hist',
    random_state=42,
    n_jobs=4
)

xgb_model.fit(X_res, y_res)
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
# Probabilités (classe "1")
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

#  AUC — ne dépend pas du seuil
auc = roc_auc_score(y_test, y_pred_proba)
print(f" AUC sur le test set : {auc:.3f}")

#  Application du seuil personnalisé
threshold = 0.3  # plus sensible pour détecter les "1"
y_pred_class = (y_pred_proba >= threshold).astype(int)

#  Matrice de confusion
cm = confusion_matrix(y_test, y_pred_class)
print("\n Matrice de confusion :\n", cm)

#  Rapport de classification
cr = classification_report(y_test, y_pred_class)
print("\n Rapport de classification :\n", cr)

🏁 AUC sur le test set : 0.755

📊 Matrice de confusion :
 [[55505  1033]
 [ 4316   649]]

📄 Rapport de classification :
               precision    recall  f1-score   support

           0       0.93      0.98      0.95     56538
           1       0.39      0.13      0.20      4965

    accuracy                           0.91     61503
   macro avg       0.66      0.56      0.57     61503
weighted avg       0.88      0.91      0.89     61503



In [ ]:
# Création du dashboard interne
# ============================
# "Yes" = éligible au crédit
y_pred_class_labels = np.where(y_pred_class == 0, "Yes", "No")

dashboard_df = pd.DataFrame({
    "SK_ID_CURR": dt1.loc[y_test.index, 'SK_ID_CURR'],
    "Score_Eligibilite": y_pred_proba,
    "Prediction": y_pred_class_labels
})

print(dashboard_df.head())


cat_cols = [
    'CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_EDUCATION_TYPE',
    'OCCUPATION_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY'
]

num_cols = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
    'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
    'CNT_CHILDREN', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'
]


all_cols = cat_cols + num_cols
missing_cols = [c for c in all_cols if c not in data.columns]
if missing_cols:
    print(f" Colonnes manquantes dans 'data' : {missing_cols}")


cols_to_add = [c for c in all_cols if c in data.columns]
dashboard_df = pd.concat([dashboard_df, data.loc[y_test.index, cols_to_add]], axis=1)

#  Sauvegarde CSV
output_path = "C:/Users/fbe/Desktop/Bloc 4/dashboard_eligibilite2.csv"
dashboard_df.to_csv(output_path, sep=";", index=False)
print(f" Fichier exporté : {output_path}")

        SK_ID_CURR  Score_Eligibilite Prediction
256571      396899           0.060965        Yes
191493      322041           0.058236        Yes
103497      220127           0.332005         No
130646      251531           0.039546        Yes
211898      345558           0.103620        Yes
✅ Fichier exporté : C:/Users/fbe/Desktop/Bloc 4/dashboard_eligibilite2.csv


In [ ]:
import joblib


encoded_cols = encoder.get_feature_names_out(cat_cols)
all_final_cols = list(num_cols) + list(encoded_cols)

#  Sauvegarde du modèle
joblib.dump(xgb_model, "modele_xgboost_eligibilite_credit.pkl")
joblib.dump(encoder, "encodeur_variables_categorielles.pkl")
joblib.dump(all_final_cols, "colonnes_modele.pkl")

print(f" Modèle XGBoost, encodeur et colonnes sauvegardés avec succès !")
print(f" Nombre total de colonnes : {len(all_final_cols)}")

✅ Modèle XGBoost, encodeur et colonnes sauvegardés avec succès !
📊 Nombre total de colonnes : 43


In [21]:
cols = joblib.load("colonnes_modele.pkl")
print(len(cols), "colonnes attendues :")
print(cols)

43 colonnes attendues :
['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'CNT_CHILDREN', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'CODE_GENDER_M', 'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Unknown', 'NAME_FAMILY_STATUS_Widow', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher', 'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Secondary / secondary special', 'OCCUPATION_TYPE_Cleaning staff', 'OCCUPATION_TYPE_Cooking staff', 'OCCUPATION_TYPE_Core staff', 'OCCUPATION_TYPE_Drivers', 'OCCUPATION_TYPE_HR staff', 'OCCUPATION_TYPE_High skill tech staff', 'OCCUPATION_TYPE_IT staff', 'OCCUPATION_TYPE_Laborers', 'OCCUPATION_TYPE_Low-skill Laborers', 'OCCUPATION_TYPE_Managers', 'OCCUPATION_TYPE_Medicine staff', 'OCCUPATION_TYPE_Private service staff', 'OCCUPATIO